In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Set random seed for numpy
np.random.seed(11)
# Set random seed for pytorch
torch.manual_seed(11)
# Set random seed for pytorch running in GPU
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(11)

In [ ]:
USC_train_features = np.load("usc_multi/train_features_USC_mod_al_32.npy")
USC_test_features = np.load("usc_multi/test_features_USC_mod_al_32.npy")
USC_train_labels = np.load("usc_multi/train_labels_USC_mod_al_32.npy")
USC_test_labels = np.load("usc_multi/test_labels_USC_mod_al_32.npy")

In [ ]:
USC_train_features = np.load("usc_single/train_features_USC_128.npy") ### <- change file path
USC_test_features = np.load("usc_single/test_features_USC_128.npy")
USC_train_labels = np.load("usc_single/train_labels_USC_128.npy")
USC_test_labels = np.load("usc_single/test_labels_USC_128.npy")

In [ ]:
# Hyperparameters
num_epochs = 15
batch_size = 10
learning_rate = 0.0001

In [ ]:
USC_train_labels = USC_train_labels.astype(int) - 1
USC_test_labels = USC_test_labels.astype(int) - 1

In [ ]:
# Transform to torch tensor
tensor_USC_train_features = torch.Tensor(USC_train_features) 
tensor_USC_train_labels = torch.Tensor(USC_train_labels)
tensor_USC_test_features = torch.Tensor(USC_test_features) 
tensor_USC_test_labels = torch.Tensor(USC_test_labels)
# Add one dimension of channel
tensor_USC_train_features = torch.unsqueeze(tensor_USC_train_features, 1)
tensor_USC_test_features = torch.unsqueeze(tensor_USC_test_features, 1)
# Create datset
train_dataset_USC = TensorDataset(tensor_USC_train_features,tensor_USC_train_labels)
test_dataset_USC = TensorDataset(tensor_USC_test_features,tensor_USC_test_labels)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset_USC, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset_USC, batch_size=batch_size, shuffle=False)

In [ ]:
for image,label in train_loader:
    print(image.size())
    print(label.size())
    break

In [ ]:
print(len(train_loader.dataset))
print(len(test_loader.dataset))

### Construct LSTM

In [ ]:
### CNN-LSTM

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
inputs = torch.rand(10,144)
mod = nn.Linear(144,12)
out = mod(inputs)
print(out.shape)

In [ ]:
inputs = torch.rand(10,64,9,192)
print(inputs.view(10,128,-1).size())

In [ ]:
class Cnn(nn.Module): ### CNN model
    def __init__(self):
        super(Cnn, self).__init__()
        self.conv1 = nn.Conv2d(1,32, kernel_size=5, stride=1, padding=2)  ### 1st conv layer
        self.pool = nn.MaxPool2d(2, 2) ### max pool layer
        self.conv2 = nn.Conv2d(32,64, kernel_size=5, stride=1, padding=2) ### 2nd conv layer
    
    def forward(self, x):
        batch,c,h,w = x.size()
        x = self.pool(F.relu(self.conv1(x))) 
        x = (F.relu(self.conv2(x))) 
        x = x.view(batch,-1) 
        return x

In [ ]:
class CNN_LSTM(nn.Module):  ### CNN-LSTM model
    def __init__(self):
        super(CNN_LSTM, self).__init__() ###(num_samples,timesteps,input_dim). 
        self.cnn = Cnn() 
        self.rnn = nn.LSTM( ###[batch_size,seq_len, nb_features]
            input_size=288, 
            hidden_size=144,
            num_layers=3, ### 3 LSTM layers
            batch_first=True) 
        self.linear = nn.Linear(144,12) ### 12 activities for USC data set

    def forward(self, x):
        batch,c,h,w = x.size()
        c_out = self.cnn(x)
        r_in = c_out.view(batch,128,-1)
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        return F.log_softmax(r_out2, dim=1)


In [ ]:
# Choose CPU or GPU
# model = ConvNet() #CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Combine

In [ ]:
%%time
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.to(device).long())
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

In [ ]:
classes = ('WALKING_F','WALKING_L','WALKING_R','WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS',"Running_F",
           "Jumping_up",'SITTING', 'STANDING', 'Sleeping',"Elevator_up","Elevator_down")

In [ ]:

from sklearn import metrics
import seaborn as sns
# Test the model on GPU
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.append(predicted)
        all_labels.append(labels)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

    print('Test Accuracy of the model on the UCIHAR test images: {} %'.format((correct / total) * 100))

mat = metrics.confusion_matrix(torch.cat(all_predictions).cpu(), torch.cat(all_labels).cpu())
plt.figure(figsize=(7,7))
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=classes, yticklabels=classes)
plt.xlabel('true label')
plt.ylabel('predicted label');